In [ ]:
import re
import pandas as pd
import pdfplumber
import os
from pathlib import Path  
from openpyxl import load_workbook
import sys
import pdfplumber
from PIL import Image
from pytesseract import pytesseract
from pdf2image import convert_from_path
import random
import datetime
pytesseract.tesseract_cmd = r'.\Tesseract-OCR\tesseract.exe'


In [ ]:

col_names =  ['Chave','Trecho_Busca_txt']
now = datetime.datetime.now()
compl = str(now.day).zfill(2) + str(now.month) +"_"+ str(now.hour) + str(now.minute)
my_df  = pd.DataFrame(columns = col_names)
my_df.to_excel('./Leitura_Geral_'+ compl + '.xlsx',index = False,header= True)

options = {}
options['strings_to_formulas'] = False
options['strings_to_urls'] = False

path = './Leitura_Geral_'+ compl + '.xlsx'
book = load_workbook(path)
writer = pd.ExcelWriter('./Leitura_Geral_'+ compl + '.xlsx', engine='openpyxl', options=options)
writer.book = book
writer.sheets = {ws.title: ws for ws in book.worksheets}
all_text = ''
Busca_txt = ''
#df = pd.DataFrame({'Chave' : '', 'Texto': [all_text],'Busca_txt': [Busca_txt]})
#df

#função de extrair e ler imagem


In [ ]:

def extrair_imagem(pag):
    saida = './imagem.jpg'
    pages = convert_from_path(arq, 400,first_page=pag+1,last_page = pag+1,poppler_path = "./poppler-21.03.0/Library/bin")
    
    for page in pages:
        page.save(saida, 'JPEG')
    #pages.close()
    text = ""
    texto_bruto = pytesseract.image_to_string(Image.open(saida))
    #os.remove(filename)
    #print(text)

    lines = texto_bruto.split("\n")
    non_empty_lines = [line for line in lines if line.strip() != ""]

    string_without_empty_lines = ""
    for line in non_empty_lines:
        text += line + "\n"
    return text



In [ ]:
    
user_direc = os.path.abspath(os.curdir) + '\pdfs\\' 
directory = os.fsencode(user_direc)
print(directory)
apendou = "nao"
x = 1
for file in os.listdir(directory):
    try:
        filename = os.fsdecode(file)
        arq = str(directory, 'utf-8') + str(file, 'utf-8')
        #print(arq)
        print('Lendo arquivo '+str(x) + ' - '+ str(file, 'utf-8'))
        x+=1
        if filename.endswith('.pdf'):

            #total_pages = len(pdf.pages)
            with pdfplumber.open(arq) as pdf:

                        # page = pdf.pages[0] - comment out or remove line
                        # text = page.extract_text() - comment out or remove line
                all_text = ''
                pag = 0
                totalpages = len(pdf.pages)
                for pdf_page in range(pag,totalpages):
                    single_page_text = pdf.pages[pdf_page].extract_text()
                    #print(pag)
                    #print(single_page_text)
                    if str(single_page_text) == 'None':
                        single_page_text = extrair_imagem(pag) 
                # separate each page's text with newline
                    if len(all_text)<30000:
                        all_text = all_text + single_page_text.replace('\n','')
                    else:
                        #all_text = all_text + '\n' + single_page_text
                        break
                    pag +=1
                    pdf.pages[pdf_page].flush_cache()
            Chave = (str(file, 'utf-8'))[0:23].replace('_','/')
            #all_text = re.sub("\s\s+", " ", all_text).replace('\n','').replace('\t','').replace(';','')  
            pdf.pages[pdf_page].flush_cache()
            #print(all_text)
            #print(len(all_text))
            t_revel = re.compile("(.{50} text1.{50}|.{50} text1_variacao .{50})")
            
            if t_revel.search(all_text) is None:
                Busca_txt = ''
            else:
                Busca_txt = '(...)'+  t_revel.search(all_text).group(0)+'(...)'
            
            df = pd.DataFrame({'Chave' : Chave,'Trecho_Busca_txt':[Busca_txt]})
            
            #df['Trecho_Busca_txt'] = df['Texto'].str.extract(r"(.{50} text1.{50}|.{50} text1_variacao .{50})", expand=True)
            
 #Busca_txt = df[(df.Trecho_Busca_txt.notnull())]
            
            #df.append(df2)
            #with open(r'.\fileName.csv', 'a') as f:
                #f.write(all_text)
                    #export
            #exporta somente quem tem Busca_txt
            if Busca_txt != '':  
                df.to_excel(writer, startrow=writer.sheets['Sheet1'].max_row, index = False,header= False)
                writer.save()
                
            all_text = ""
            Busca_txt = ''
        pdf.close()

       # append_df_to_excel('./aa.xlsx', df2,header=None, index=False)  
    except Exception as e:
        print(e)
        pdf.close()
        pass
#f.close()
pdf.close()
writer.close()

Busca_txts_df = pd.read_excel('./Busca_txts_app.XLSX')

df = pd.read_excel('./Leitura_Geral_'+ compl + '.xlsx')
Busca_txts_df.head()
df_nova = df.merge(Busca_txts_df, left_on='Chave', right_on='Chave', how='left')
df_nova
df_nova = df_nova[df_nova['Busca_txt_evento'].isnull()]
df_nova


if df_nova.empty:
    print('Não há novas Busca_txts identificadas')
else:
    df_nova.to_excel('./Busca_txts_final_'+str(now.day).zfill(2) + str(now.month) +"_"+ str(now.hour) + str(now.minute) + '.xlsx')
